In [1]:
# Step 1: Install dependencies
!pip install flask flask-ngrok pyngrok diffusers transformers accelerate scipy safetensors --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00


In [2]:
# Step 2: Import libraries
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import io, base64


In [3]:
# Step 3: Load Stable Diffusion model
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

print("✅ Model loaded on", device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Model loaded on cuda


In [4]:
# Step 4: Create Flask app
app = Flask(__name__)

HTML = """
<!DOCTYPE html>
<html>
<head><title>AI Art Generator</title></head>
<body>
    <h2>🎨 AI Art Generator</h2>
    <form method="POST">
        <input name="prompt" placeholder="Enter prompt" style="width:300px;" required>
        <button type="submit">Generate</button>
    </form>
    {% if img_data %}
        <h3>Prompt: {{ prompt }}</h3>
        <img src="data:image/png;base64,{{ img_data }}" width="512"/>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def home():
    img_data = None
    prompt = None
    if request.method == "POST":
        prompt = request.form["prompt"]
        prompt = f"{prompt}, highly detailed, trending on artstation"

        with torch.autocast("cuda" if device == "cuda" else "cpu"):
            image = pipe(prompt).images[0]

        # Convert to base64 for display
        buffer = io.BytesIO()
        image.save(buffer, format="PNG")
        img_data = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return render_template_string(HTML, img_data=img_data, prompt=prompt)


In [5]:
!ngrok config add-authtoken 2yGoKhUCLFBfu8kO3lfosPkZvEn_CdpzBrjUuo5SmxCSq3od


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Step 5: Start server with pyngrok
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)

app.run(port=5000)


🚀 Public URL: NgrokTunnel: "https://4586-34-125-155-219.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 09:07:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 09:07:05] "GET /favicon.ico HTTP/1.1" 404 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 09:07:45] "POST / HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 09:08:21] "POST / HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 09:09:02] "POST / HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [24/Jun/2025 09:09:25] "POST / HTTP/1.1" 200 -
